In [395]:
import ccxt
import os
from dotenv import load_dotenv
import time
import pandas as pd
import numpy as np
from datetime import date, datetime, timezone, tzinfo

In [396]:
load_dotenv()

# Retrieve API keys from environment variables
xP_KEY = os.getenv('xP_KEY')
xP_SECRAT = os.getenv('xP_SECRET')

kC_KEY = os.getenv('kC_KEY')
kC_SECRET = os.getenv('kC_SECRAT')
kC_PASSWORD = os.getenv('kC_PASSWORD')


In [397]:
# Initialize the KuCoin exchange object
kucoin = ccxt.kucoin({
    'enableRateLimit': True,
    'apiKey': kC_KEY,
    'secret': kC_SECRET,
    'password': kC_PASSWORD
})
phemex = ccxt.phemex({
    'enableRateLimit': True,
    'apiKey': xP_KEY,
    'secret': xP_SECRAT
})
symbol = 'BTCUSD'

In [398]:
def ask_bid():
    order_book = phemex.fetch_order_book(symbol=symbol)
    return order_book['asks'][0][0], order_book['bids'][0][0]

# Dialy SMA 20 Days

In [399]:
 #! FIND DAILY SMA PER 20 DAYS

def sma_20d():
    print("starting 20 Days ...")

    timeframe = '1d'
    num_bars = 100
    bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)
    # print(" Got the Bars: ",bars)
    data_20days = pd.DataFrame(bars, columns=['timestep', 'open', 'high', 'low', 'close', 'volume'])
    data_20days['date'] = pd.to_datetime(data_20days['timestep'], unit='ms')
    data_20days.drop(columns=['timestep'], inplace=True)
    data_20days['SMA_20'] = data_20days['close'].rolling(window=20).mean()

    bid = ask_bid()[1] 
    data_20days.loc[data_20days['SMA_20'] > bid, 'SIGNAL'] = 'SELL'
    data_20days.loc[data_20days['SMA_20'] < bid, 'SIGNAL'] = 'BUY'

    if data_20days['SIGNAL'].iloc[-1] == 'SELL':
        print("Opening as sell ...")
    else:
        print('Opening as Buy ...')
    
    return data_20days

sma_20d()


starting 20 Days ...
Opening as Buy ...


,open,high,low,close,volume,date,SMA_20,SIGNAL
0,64850.7,65048.8,63352.2,64096.5,102192438.0,2024-06-21,NaN,NaN
1,64111.9,64484.4,63919.6,64224.3,12399096.0,2024-06-22,NaN,NaN
2,64220.5,64475.0,63082.3,63133.5,34814744.0,2024-06-23,NaN,NaN
3,63139.7,63290.0,58206.9,60223.7,232487908.0,2024-06-24,NaN,NaN
4,60220.8,62372.2,60193.2,61761.2,80158692.0,2024-06-25,NaN,NaN
...,...,...,...,...,...,...,...,...
95,63412.0,64744.2,62750.0,64316.1,153646836.0,2024-09-24,59550.335,BUY
96,64316.2,64917.0,62900.0,63119.1,106151622.0,2024-09-25,59895.090,BUY
97,63126.8,65843.1,62650.0,65175.6,126242249.0,2024-09-26,60451.990,BUY
98,65175.6,66490.8,64828.6,65742.1,97440115.0,2024-09-27,61027.830,BUY


#  15 Minutes SMA

In [400]:
 #! Define the SMA for the 15 Minutes With Prices bp_1, bp_2, sp_1, sp_2

def sma_15min():
    print("starting sma 15 Minuts  ...")

    timeframe = '15m'
    num_bars = 100
    bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)
    # # print(" Got the Bars: ",bars)
    data_15m = pd.DataFrame(bars, columns=['timestep', 'open', 'high', 'low', 'close', 'volume'])
    data_15m['date'] = pd.to_datetime(data_15m['timestep'], unit='ms')
    data_15m.drop(columns=['timestep'], inplace=True)
    data_15m['SMA_15m'] = data_15m['close'].rolling(window=20).mean()

    bid = ask_bid()[1] 
        
    data_15m['bp_1'] = data_15m['SMA_15m'] * 1.001
    data_15m['bp_2'] = data_15m['SMA_15m'] * 0.997
    data_15m['sp_1'] = data_15m['SMA_15m'] * 0.999
    data_15m['sp_2'] = data_15m['SMA_15m'] * 1.003

    return data_15m

sma_15min()

starting sma 15 Minuts  ...


,open,high,low,close,volume,date,SMA_15m,bp_1,bp_2,sp_1,sp_2
0,65686.4,65686.4,65625.4,65651.4,11701.0,2024-09-28 12:15:00,NaN,NaN,NaN,NaN,NaN
1,65643.8,65643.8,65387.2,65437.7,822651.0,2024-09-28 12:30:00,NaN,NaN,NaN,NaN,NaN
2,65465.4,65542.5,65463.9,65538.4,102612.0,2024-09-28 12:45:00,NaN,NaN,NaN,NaN,NaN
3,65538.4,65541.8,65439.5,65454.4,293387.0,2024-09-28 13:00:00,NaN,NaN,NaN,NaN,NaN
4,65454.3,65546.8,65403.7,65546.6,445696.0,2024-09-28 13:15:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,65682.8,65749.9,65682.8,65704.9,92476.0,2024-09-29 12:00:00,65570.545,65636.115545,65373.833365,65504.974455,65767.256635
96,65705.6,65745.0,65666.1,65667.0,78355.0,2024-09-29 12:15:00,65579.350,65644.929350,65382.611950,65513.770650,65776.088050
97,65667.0,65711.0,65660.7,65708.1,57588.0,2024-09-29 12:30:00,65590.020,65655.610020,65393.249940,65524.429980,65786.790060
98,65708.1,65708.3,65618.9,65620.1,132580.0,2024-09-29 12:45:00,65598.000,65663.598000,65401.206000,65532.402000,65794.794000


# Bot

In [438]:
def bot():
    data_15m = sma_15min()
    data_20d = sma_20d()
    ask,bid = ask_bid()

    # * Define OPEN Order
    signal = data_20d.iloc[-1]['SIGNAL']
    
    open_size = 1
    
    in_pos = pnl_close()[1]


    if not in_pos:
        if signal == "BUY":
            print("Make order as BUY !!")
            bp_1 = data_15m.iloc[-1]['bp_1']
            bp_2 = data_15m.iloc[-1]['bp_2']
            print(f"bp_1: {bp_1} bp_2: {bp_2}")
            # TODO: Make Buy Order Here ...
        else:
            print("Make order as SELL !!")
            sp_1 = data_15m.iloc[-1]['sp_1']
            sp_2 = data_15m.iloc[-1]['sp_2']
            print(f"sp_1: {sp_1} sp_2: {sp_2}")
            # TODO: Make Sell Order Here ... 
    else:
        print("Already In Position, not making new Position")
bot()

starting sma 15 Minuts  ...
starting 20 Days ...
Opening as Buy ...
Checking to see if it's time to exit...
Already In Position, not making new Position


In [440]:
target = 25

def kill_switch():
    print("Starting the Kill switching ...")
    # TODO: Implement the Kill Switch
    pass

def pnl_close_phemex():
    print("🔍 Checking to see if it's time to exit on Phemex...")
    
    # Example of different parameters
    params = {'type': 'future', 'currency': 'BTC'}
    poc_dic = phemex.fetch_positions(params=params)[0]
    side = poc_dic['side']
    entry_price = float(poc_dic['info']['avgEntryPrice'])
    leverage = float(poc_dic['leverage'])
    current_price = ask_bid()[1]

    if side == 'long':
        diff = current_price - entry_price
    else:
        diff = entry_price - current_price
    
    try:
        perc = ((diff / entry_price) * leverage)
    except ZeroDivisionError:
        perc = 0

    return f"📊 Difference is {diff} and Percentage: {perc:.2f}%"

def pnl_close_kucoin():
    print("Starting PNL ...")
    
    # Example of different parameters

    poc_dic = kucoin.fetch_ticker(symbol='STRK/USDT')
    perc = poc_dic['percentage']
    diff = poc_dic['info']['changePrice']

    print(f"Difference is {diff} and Percentage: {perc}%")

    pnlclose = False
    in_pos = False
    if perc > 0:
        # * We are in a winning position (profit)
        if perc > target:
            # ! Closing the position as target is hit
            print("In Profit also Hit the Target ...")
            pnlclose = True
            kill_switch()
        else:
            print("In Profit but  not hit the target, so wait !!")
    else:
        in_pos = True
        print("We are not in Profit yet.")
    print("Finishing the PNL close ...")

    return pnlclose, in_pos

# print(pnl_close_phemex())
pnl_close_kucoin()


Starting PNL ...
Difference is 0.00086 and Percentage: 0.18%
In Profit but  not hit the target, so wait !!
Finishing the PNL close ...


(False, False)